Oliwer & Mohand - CartPole V1

In [25]:
import numpy as np
import gymnasium as gym
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard
from keras.models import Sequential
from keras.layers import Dense
import random
from collections import deque

In [26]:
# Variables
env_name = 'CartPole-v1' # Nom de l'environnement
epsilon = 1.0 # Valeur initiale d'epsilon (exploration)
epsilon_min = 0.01 # Valeur minimale d'epsilon
epsilon_decay = 0.99 # Facteur de décroissance d'epsilon
gamma = 0.99 # Eviter que ça mange comme un doberman (Entraienment sur le long terme)
batch_size = 32 # Taille de l'entraienement (données mémoire)
memory_size = 1000000 # Taille de la mémoire 
episodes = 500 # Nombre d'épisodes d'entraînement (itérations)

In [27]:
# Environnement
env = gym.make(env_name) # Initialisation de l'environnement (Pendulum-v1)
state_shape = env.observation_space.shape[0] # Récupération de l'état de l'environnement
action_shape = env.action_space.n # Récupération de l'action de l'environnement

state_shape, action_shape

(4, np.int64(2))

In [28]:
# Callbacks 
callbacks = [
    #ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6),
    #EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True),
    #ModelCheckpoint(filepath='modelSigmoid.keras', save_best_only=True),
]

In [29]:
# Création du modèle

def create_q_model():
    model = Sequential([
        Dense(64, input_shape=(state_shape,), activation='relu'),
        Dense(64, activation='relu'),
        Dense(action_shape, activation='linear')
    ])
    model.compile(optimizer="adam", loss="mse")
    return model

In [30]:
# Initialisation des modèles

q_model = create_q_model() # Création du modèle
target_model = create_q_model() # Création du modèle cible
target_model.set_weights(q_model.get_weights()) # Initialisation du modèle cible avec les poids du modèle principal

In [31]:
# Initialisation de la mémoire

memory = deque(maxlen=memory_size)

In [32]:
# Fonction de stockage de la mémoire

def store_transition(state, action, reward, next_state, terminated):
    memory.append((state, action, reward, next_state, terminated))

In [33]:
# Echantillonnage du batch

def sample_batch(): 
    batch = random.sample(memory, batch_size) # Echantillonnage aléatoire du batch
    state, action, reward, next_state, terminated = map(np.asarray, zip(*batch))
    return np.array(state), np.array(action), np.array(reward), np.array(next_state), np.array(terminated)

In [34]:
# Epsilon-greedy policy

def epsilon_greedy_policy(state, epsilon):
    if np.random.random() < epsilon: # Exploration (Si le nombre aléatoire est inférieur à epsilon, on choisi une action aléatoire)
        return np.random.choice(action_shape)
    else: # Si non, on utilise le modèle pour prédire l'action
        q_values =  q_model.predict(state[np.newaxis], verbose=0) 
        return np.argmax(q_values[0]) 

In [35]:
# Entrainement

def train_step():
    if len(memory) < batch_size: # Si la mémoire est inférieure à la taille du batch, on ne fait rien
        return
    
    state, action, reward, next_state, terminated = sample_batch() # Echantillonnage du batch
    
    next_q_values = target_model.predict(next_state, verbose=0) # Prédiction des valeurs Q de l'état suivant
    max_next_q_values = np.max(next_q_values, axis=1) # Récupération de la valeur maximale des valeurs Q de l'état suivant

    target_q_values = q_model.predict(state, verbose=0) # Prédiction des valeurs Q de l'état actuel

    for i, act in enumerate(action):
        target_q_values[i][act] = reward[i] if terminated[i] else reward[i] + gamma * max_next_q_values[i]

    q_model.fit(state, target_q_values, epochs=1, verbose=0, callbacks=callbacks) # Entraînement du modèle sur l'état actuel et les valeurs Q cibles

In [36]:
reward_history = [] # Historique des récompenses

In [37]:
# Itération des entrainements

for episode in range(episodes):
    state, _ = env.reset()
    total_reward = 0
    terminated = False
    
    while not terminated:
        action = epsilon_greedy_policy(state, epsilon) # Sélection de l'action avec la politique epsilon-greedy
        next_state, reward, terminated, truncated, _ = env.step(action) # Modifier l'environnement avec l'action
        terminated = terminated or truncated # Vérification de la terminaison de l'épisode
        store_transition(state, action, reward, next_state, terminated) # Stockage de la transition dans la mémoire
        total_reward += reward # Ajout de la récompense totale

        state = next_state # Mise à jour de l'état
        train_step() # Entraînement du modèle

    epsilon = max(epsilon_min, epsilon * epsilon_decay) # Décroissance d'epsilon (réduction de l'exploration)

    if episode % 10 == 0:
        target_model.set_weights(q_model.get_weights()) # Mise à jour du modèle cible avec les poids du modèle principal
    
    reward_history.append(total_reward) # Ajout de la récompense totale à l'historique des récompenses
    print(f"Episode {episode + 1}/{episodes}, Total Reward: {total_reward}, Epsilon: {epsilon:.4f}")

Episode 1/500, Total Reward: 40.0, Epsilon: 0.9900
Episode 2/500, Total Reward: 15.0, Epsilon: 0.9801


KeyboardInterrupt: 

In [ ]:
# Sauvegarde du modèle
target_model.save('CartPole_model.keras') # Sauvegarde du modèle cible